# Jupyter Notebook for `train_model.py`
This notebook explains and executes the logic of the `train_model.py` script. Each section will describe the purpose of the code and walk through the implementation.

## Imports and Setup
These imports include FastAI for vision tasks, PyTorch for model and loss definitions, and utility modules like `sklearn` and `collections` for metrics and data handling. Also the path for the training and validation dataset is defined.

In [ ]:
from fastai.vision.all import *
import multiprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from fastai.learner import Metric
import numpy as np
from collections import Counter
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from fastai.learner import Metric
from collections import defaultdict

data_path = Path("../../images/dataset")

## `BalancedLabelDistanceLoss` Class
This custom loss penalizes predictions based on the distance between predicted and true class indices, weighted by class frequency to mitigate class imbalance.

In [ ]:

class BalancedLabelDistanceLoss(nn.Module):
    def __init__(self, class_counts, epsilon=1e-6):
        super().__init__()
        self.register_buffer('weights', 1.0 / (torch.tensor(class_counts).float() + epsilon))
        self.weights = self.weights / self.weights.sum() * len(class_counts)

    def forward(self, preds, targets):
        probs = F.softmax(preds, dim=1)
        class_indices = torch.arange(preds.size(1), device=preds.device).float()
        expected = (probs * class_indices).sum(dim=1)

        error = (expected - targets.float()).pow(2)

        sample_weights = self.weights[targets]
        return error * sample_weights


## `BalancedHybridLoss` Class
Combines CrossEntropyLoss and the BalancedLabelDistanceLoss using a weighted alpha parameter. Helps to better capture both classification accuracy and class-distance sensitivity.

In [ ]:

class BalancedHybridLoss(nn.Module):
    def __init__(self, class_counts, alpha=0.8):
        super().__init__()
        self.alpha = alpha
        self.ce = nn.CrossEntropyLoss(reduction='none')
        self.label_distance = BalancedLabelDistanceLoss(class_counts)

    def forward(self, preds, targets, **kwargs):
        loss_ce = self.ce(preds, targets)
        loss_ld = self.label_distance(preds, targets)
        combined = self.alpha * loss_ce + (1 - self.alpha) * loss_ld

        if kwargs.get('reduction', None) == 'none':
            return combined
        else:
            return combined.mean()


## `BalancedLabelDistance` Metric
A custom metric to evaluate the average distance between predicted and actual labels per class. These average distances per class are added together to also get the average across all classes.

In [ ]:

class BalancedLabelDistance(Metric):
    def __init__(self): self._name = 'balanced_label_distance'

    def reset(self):
        self.class_totals = defaultdict(float)
        self.class_counts = defaultdict(int)

    def accumulate(self, learn):
        preds = learn.pred.argmax(dim=1)
        targets = learn.y

        for pred, target in zip(preds, targets):
            error = abs(pred.item() - target.item())
            self.class_totals[target.item()] += error
            self.class_counts[target.item()] += 1

    @property
    def value(self):
        class_avg_distances = []
        for cls in sorted(self.class_counts.keys()):
            count = self.class_counts[cls]
            if count > 0:
                avg = self.class_totals[cls] / count
                class_avg_distances.append(avg)
        return sum(class_avg_distances) / len(class_avg_distances) if class_avg_distances else None


## Detailed Explanation of the main routine

This section orchestrates the entire model training and evaluation pipeline using the FastAI library. 

### 1. Data Loading and Transformation
Purpose: Load images from `train/` and `val/` folders and apply preprocessing.

- Resize all images to 224x224
- Apply image augmentations like flipping, zoom, warp, etc.
- Normalize with ImageNet statistics
- Use a batch size of 64
- Shuffle training data for better generalization

In [ ]:
multiprocessing.freeze_support()

dls = ImageDataLoaders.from_folder(
        data_path,
        train='train',
        valid='val',
        valid_pct=None,
        item_tfms=Resize(224),
        batch_tfms=aug_transforms(mult=1.5) + [Normalize.from_stats(*imagenet_stats)],
        bs=64,
        num_workers=0,
        shuffle=True
    )
print(f"Number of training batches: {len(dls.train)}")   

### 2. Compute Class Counts
Purpose: Count how many training samples exist for each class. This helps to create a balanced loss function so rare classes are not ignored.

- Extract labels from folder names
- Count number of samples per label
- Fill in missing classes with count 1 to avoid division by zero
- Store this in `class_counts`

In [ ]:
train_folder = data_path / 'train'
train_labels = [int(path.parent.name) for path in train_folder.rglob('*') if path.is_file()]
num_classes = len(set(train_labels))
counts = Counter(train_labels)
class_counts = [counts.get(i, 1) for i in range(num_classes)]
print("Class counts:", class_counts)

### 3. Create Learner with Custom Loss and Metric
Purpose: Set up the model training configuration using FastAI's Learner.

- `resnet50`: Pretrained model used as a backbone
- `BalancedHybridLoss`: Custom loss combining cross-entropy and label distance
- `BalancedLabelDistance`: Custom metric tracking prediction distance from true labels
- Callbacks:
  - SaveModelCallback: Save best-performing model
  - EarlyStoppingCallback: Stop early if no improvement
  - ReduceLROnPlateau: Lower LR if validation loss stalls
  - GradientClip: Prevent exploding gradients
  - CSVLogger: Log metrics to CSV

In [ ]:
learn = vision_learner(dls, resnet50,
    loss_func=BalancedHybridLoss(class_counts, alpha=0.35),
    metrics=[
        accuracy,
        BalancedLabelDistance()
    ],
    cbs=[
        SaveModelCallback(monitor='balanced_label_distance', comp=np.less, fname='best_model'),
        EarlyStoppingCallback(monitor='valid_loss', comp=np.less, patience=5),
        ReduceLROnPlateau(monitor='valid_loss', patience=2),
        GradientClip(max_norm=1.0),
        CSVLogger(fname='training_log.csv')
    ])
print("Created learner")

### 4. Train and Fine-Tune Model
Purpose: Fine-tune the model over 40 epochs. Uses transfer learning.

- First few epochs train the head only
- Then all layers are unfrozen and trained

In [ ]:
learn.fine_tune(40)
print("Fine-tuned model")

### 5. Evaluate Overall Accuracy
Purpose: Check how well the model performs overall on validation data.

- `get_preds()` returns predictions and labels
- `accuracy_score` gives overall classification accuracy

In [ ]:
val_predictions, val_targets = learn.get_preds()
val_accuracy = accuracy_score(val_targets, val_predictions.argmax(dim=1))
print(f"\nValidation Accuracy: {val_accuracy:.4f}")

### 6. Balanced Label Distance Evaluation
Purpose: Measure the average distance between predicted and actual class indices.

- For example, predicting class 4 when the true label is 5 is less bad than predicting class 1
- This metric gives a more nuanced view of performance, especially in ordinal problems

In [ ]:
class_totals = defaultdict(float)
class_counts = defaultdict(int)
for pred, target in zip(val_predictions.argmax(dim=1), val_targets):
    dist = abs(pred.item() - target.item())
    class_totals[target.item()] += dist
    class_counts[target.item()] += 1
class_avg_dists = []
for cls in sorted(class_counts.keys()):
    avg_dist = class_totals[cls] / class_counts[cls]
    class_avg_dists.append(avg_dist)
    print(f"Class {cls}: Avg label distance = {avg_dist:.4f}")
balanced_distance = sum(class_avg_dists) / len(class_avg_dists)
print(f"\nBalanced Label Distance (validation): {balanced_distance:.4f}")

### 7. Confusion Matrix and Per-Class Accuracy
Purpose: Visualize which classes are confused with each other and calculate per-class accuracy.

- Confusion matrix shows counts of true vs predicted labels
- Per-class accuracy helps identify which labels are hardest to predict

In [ ]:
preds, targs = learn.get_preds(dl=learn.dls.valid)
decoded = preds.argmax(dim=1)
assert len(decoded) == len(targs), f"Shape mismatch: {len(decoded)} vs {len(targs)}"
cm = confusion_matrix(targs.cpu(), decoded.cpu())
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=learn.dls.vocab)
disp.plot(xticks_rotation=45, cmap='Blues')
plt.tight_layout()
plt.show()

print("\nPer-class accuracy:")
cm = confusion_matrix(val_targets, val_predictions.argmax(dim=1))
per_class_acc = cm.diagonal() / cm.sum(axis=1)
for cls, acc in zip(learn.dls.vocab, per_class_acc):
    print(f"{cls}: {acc:.4f}")
print("Evaluation completed")

### 8. Export the Trained Model
Purpose: Save the trained model to disk as a `.pkl` file.

- Can be loaded later for inference
- Includes model weights, transforms, and label mappings

In [ ]:
learn.remove_cb(CSVLogger)
learn.export("road_quality_classifier.pkl")
print("Model saved successfully")

### Summary

| Step | Purpose |
|------|---------|
| Data Loading | Load and preprocess image data |
| Class Count | Adjust loss to balance rare vs common classes |
| Learner Setup | Define model, loss, metrics, and training hooks |
| Training | Fine-tune model using transfer learning |
| Evaluation | Measure accuracy and custom distance metric |
| Visualization | See how well each class is recognized |
| Export | Save trained model for reuse |